In [2]:
import sqlite3 as sql
import pandas as pd

In [3]:
pd.set_option('display.max_rows', 500)
pd.options.display.max_rows
conn = sql.connect('librairieImmobilier.db')

cursor = conn.cursor()

# 1. Nombre total d’appartements vendus au 1er semestre 2020

In [5]:
pd.read_sql_query("SELECT COUNT(*) as 'Nombre ventes' FROM Mutation WHERE \"Nature mutation\"='Vente' AND \"Code type local\" =2 ",conn)

,Nombre ventes
0,0


# 2. Proportion des ventes d’appartements par le nombre de pièces

In [4]:
pd.read_sql_query("SELECT COUNT(*) as 'Nombre vente'\COUNT(*),\"Nombre pieces principales\" FROM Mutation INNER JOIN Batiment ON Mutation.Id = Batiment.IdMutation GROUP BY \"Nombre pieces principales\"",conn)

,Nombre vente,Nombre pieces principales
0,561360,NaN
1,192699,0.0
2,46212,1.0
3,79498,2.0
4,107553,3.0
5,111515,4.0
6,67304,5.0
7,26569,6.0
8,9714,7.0
9,3726,8.0


# 3. Liste des 10 départements où le prix du mètre carré est le plus élevé

In [5]:
pd.read_sql_query("SELECT  AVG(\"Valeur fonciere\")/AVG(\"Surface reelle bati\") as 'Prix m/2',\"Code departement\"  FROM Mutation INNER JOIN Batiment ON Mutation.Id = Batiment.IdMutation GROUP BY \"Code departement\" ORDER BY \"Prix m/2\"   LIMIT 10",conn)

,Prix m/2,Code departement
0,776.560697,23
1,811.406660,43
2,833.313873,70
3,868.734816,55
4,878.638549,52
5,1020.334146,79
6,1022.489128,8
7,1071.890994,36
8,1091.482926,15
9,1197.621014,7


# 4. Prix moyen du mètre carré d’une maison en Île-de-France

In [6]:
pd.read_sql_query("SELECT AVG(\"Valeur fonciere\")/AVG(\"Surface reelle bati\") as 'Prix m/2'FROM Mutation INNER JOIN Batiment ON Mutation.Id = Batiment.IdMutation AND \"Code departement\" IN(75,77,78,91,92,93,94,95)",conn)

,Prix m/2
0,30872.847249


# 5. Liste des 10 appartements les plus chers avec le département et le nombre de mètres carrés

In [7]:
pd.read_sql_query("SELECT  \"Valeur fonciere\",\"Code departement\",\"Surface Carrez du 1er lot\"  FROM Mutation INNER JOIN Batiment ON Mutation.Id = Batiment.IdMutation  ORDER BY \"Prix m/2\"   LIMIT 10",conn)

,Valeur fonciere,Code departement,Surface Carrez du 1er lot
0,"2175,00",1,None
1,"2175,00",1,None
2,"2175,00",1,None
3,"2175,00",1,None
4,"2175,00",1,None
5,"180300,00",1,None
6,"54800,00",1,"31,68"
7,"350750,00",1,None
8,"350750,00",1,None
9,"350750,00",1,None


# 6. Taux d’évolution du nombre de ventes entre le premier et le second trimestre de 2020

In [38]:
pd.read_sql_query("SELECT COUNT(*) FROM Mutation WHERE strftime('%m', \"Date mutation\") >= '04'  SELECT COUNT(*) FROM Mutation WHERE strftime('%m', \"Date mutation\") < '07' ",conn)

DatabaseError: Execution failed on sql 'SELECT COUNT(*) FROM Mutation WHERE strftime('%m', "Date mutation") >= '04'  SELECT COUNT(*) FROM Mutation WHERE strftime('%m', "Date mutation") < '07' ': near "SELECT": syntax error

In [51]:
pd.read_sql_query("SELECT (SELECT CAST(COUNT(*) as REAL) FROM Mutation WHERE strftime('%m', \"Date mutation\" ) < '07') / (SELECT COUNT(*) FROM Mutation WHERE strftime('%m', \"Date mutation\") >= '04') * 1.0 AS 'Evolution ventes'",conn)

,Evolution ventes
0,2.543057


# 7. Liste des communes où le nombre de ventes a augmenté d'au moins 20% entre le premier et le second trimestre de 2020

In [52]:
pd.read_sql_query("SELECT Commune FROM Batiment INNER JOIN Batiment ON Mutation.[index] = Batiment.[index] AND  WHERE (SELECT (SELECT CAST(COUNT(*) as REAL) FROM Mutation WHERE strftime('%m', \"Date mutation\" ) < '07') / (SELECT COUNT(*) FROM Mutation WHERE strftime('%m', \"Date mutation\") >= '04') * 1.0 AS 'Evolution ventes') > 1.2",conn)

DatabaseError: Execution failed on sql 'SELECT Commune FROM Batiment INNER JOIN Batiment ON Mutation.[index] = Batiment.[index] AND  WHERE (SELECT (SELECT CAST(COUNT(*) as REAL) FROM Mutation WHERE strftime('%m', "Date mutation" ) < '07') / (SELECT COUNT(*) FROM Mutation WHERE strftime('%m', "Date mutation") >= '04') * 1.0 AS 'Evolution ventes') > 1.2': near "WHERE": syntax error

# 8. Différence en pourcentage du prix au mètre carré entre un appartement de 2 pièces et un appartement de 3 pièces

# 9. Les moyennes de valeurs foncières pour le top 3 des communes des départements 6, 13, 33, 59 et 69